##  TensorFlow 高层封装之Slim, TFLearn, Keras

选择环境：Anaconda Python 3.6.6  
安装Tensorflow：Python 3.6环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 10.1 TensorFlow 高层封装总览

目前比较主流的TensorFlow高层封装主要有四个，分别是**TensorFlow-Slim、TFLearn、Keras**和**Estimator**。

**1. TensorFlow-Slim**

Google通过TensorFLow-Slim开源了一些已经训练好的图像分析的模型，所以目前在图像识别问题中TensorFLow-Slim仍被较多地使用。  
下面给出如何使用TensorFLow-Slim在MNIST数据集上实现LeNet5模型：

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data


# 通过TensorFlow-Slim来定义LeNet-5的网络结构。
def lenet5(inputs):
    # 将输入转化为一个4维数组，第一维是batch大小，后面三维表示一张图片
    inputs = tf.reshape(inputs, [-1, 28, 28, 1])
    
    # 定义第一卷积层。可以看出，TensorFLow-Slim定义的网络结构并不需要用户去关心如何声明和初始化变量，
    # 而只需要定义网络结构即可。
    net = slim.conv2d(inputs, 32, [5, 5], padding='SAME', scope='layer1-conv')
    net = slim.max_pool2d(net, 2, stride=2, scope='layer2-max-pool')
    
    # 定义第二卷积层
    net = slim.conv2d(net, 64, [5, 5], padding='SAME', scope='layer3-conv')
    net = slim.max_pool2d(net, 2, stride=2, scope='layer4-max-pool')
    
    # 直接使用TensorFLow-Slim封装好的flatten函数将4维矩阵将为2维，用户不需计算通过卷积后的矩阵大小。
    net = slim.flatten(net, scope='flatten')
    
    # 通过TensorFLow-Slim定义全连接层，500是节点数。
    net = slim.fully_connected(net, 500, scope='layer5')
    net = slim.fully_connected(net, 10, scope='output')
    
    return net


# 通过TensorFLow-Slim定义网络结构，并使用之前章节中给出的方式训练定义好的模型。
def train(mnist):
    # 1. 定义网络输入输出
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
    
    # 2. 定义前向传播、损失函数、反向传播
    y = lenet5(x)

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    loss = tf.reduce_mean(cross_entropy)

    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    # 其他准备，feed、accuracy
    feed_valid = {x:mnist.validation.images, y_:mnist.validation.labels}
    feed_test = {x:mnist.test.images, y_:mnist.test.labels}
    correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    # 3. 建立会话，训练
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(5000):
            xs, ys = mnist.train.next_batch(100)
            _, loss_value = sess.run([train_op, loss], feed_dict={x: xs, y_: ys})

            if i % 1000 == 0 or i == mnist.validation.images.shape[0] - 1:
                acc_valid = sess.run(accuracy, feed_dict=feed_valid)
                print("After %d training step(s), loss on training batch is %g, accuracy on validation is %f." % (i, loss_value, acc_valid))
                
        acc_test = sess.run(accuracy, feed_dict=feed_test)
        print('Final test accuracy is %f.' % acc_test)
                
                
def main(argv=None):
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
After 0 training step(s), loss on training batch is 2.30668, accuracy on validation is 0.095600.
After 1000 training step(s), loss on training batch is 0.651067, accuracy on validation is 0.765000.
After 2000 training step(s), loss on traini

可以看出，TensorFlow-Slim主要的作用是使模型定义更加简洁，基本上每一层网络可以通过一句话来实现。除了对单层网络结构，TensorFlow-Slim 还对数据预处理、损失函数、学习过程、测试过程等都提供了高层封装，但使用并不广泛。可参考代码库(https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim)

TensorFlow-Slim最特别的一个地方是它**对一些标准的神经网络模型进行了封装，比如VGG、Inception以及ResNet**，而且Google开源的训练好的图像分类模型基本都是通过TensorFlow-Slim实现的。通过TensorFlow-Slim开源的训练好的模型列表可以在GitHub上找到(https://github.com/tensorflow/models/tree/master/research/slim/nets)

**2. TFLearn**

通过TFLearn可以更加容易地完成模型定义、模型训练以及模型评测的全过程。TFLearn没有集成在TensorFlow地安装包中，故需要单独安装：`pip install tflearn`。下面展示如何使用TFLearn在MNIST数据集上实现LeNet5模型：

In [2]:
import tflearn
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

import tflearn.datasets.mnist as mnist

# 读取mnist数据集
trainX, trainY, testX, testY = mnist.load_data("../../datasets/MNIST_data", one_hot=True)

# 将图像数据reshape成CNN输入的格式
trainX = trainX.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

# 构建神经网络，这个过程和tensorFlow-Slim比较类似
# input_data定义了一个placeholder来接入输入数据
net = input_data(shape=[None, 28, 28, 1], name='input')
# 通过TFLearn封装好的API定义一个深度为5，过滤器为5*5，激活函数为ReLU的卷积层
net = conv_2d(net, nb_filter=32, filter_size=5, activation='relu')
# 定义了一个过滤器为2*2的最大池化层
net = max_pool_2d(net, kernel_size=2)
# 类似地定义其他地网络结构
net = conv_2d(net, 64, 5, activation='relu')
net = max_pool_2d(net, 2)
net = fully_connected(net, 500, activation='relu')
net = fully_connected(net, 10, activation='softmax')

# 使用TFLearn封装好的函数定义学习任务。指定优化器为sgd，学习率为0.01，损失函数为交叉熵
net = regression(net, optimizer='sgd', learning_rate=0.01, \
                 loss='categorical_crossentropy')

# 通过定义地网络结构训练模型，并在指定的验证集上验证模型效果。
# TFLearn将模型的训练过程封装到一个类中，这样可以减少非常多的冗余代码
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY, n_epoch=3, validation_set=([testX, testY]), show_metric=True)

Training Step: 2579  | total loss: 0.09759 | time: 177.645s
| SGD | epoch: 003 | loss: 0.09759 - acc: 0.9733 -- iter: 54976/55000
Training Step: 2580  | total loss: 0.09592 | time: 186.896s
| SGD | epoch: 003 | loss: 0.09592 - acc: 0.9729 | val_loss: 0.09362 - val_acc: 0.9705 -- iter: 55000/55000
--


可以看出，使用TFLearn训练神经网络的流程也是一样的：先定义神经网络的结构，再使用训练数据来训练模型.

**TFLearn vs. 原生态TensorFlow：**

- TFLearn不仅使神经网络结构定义更加简洁，还将模型训练的过程也进行了封装。另外，在定义神经网络的前向传播过程之后， TFLearn可以**通过regression函数来指定损失函数和优化方法**。更方便的是，**不仅TFLearn能很好地封装模型定义，tflearn.DNN也能很好地封装模型训练的过程**。通过fit函数可以指定训练中使用的数据和训练的轮数。这样避免了大量的冗余代码。

**TFLearn vs. Keras&Esitimator:**
- 相同点：封装的方式上基本一致，主要也是针对模型定义和模型训练两个部分；
- 不同点：Keras和Estimator都己经加入了TensorFlow代码库，而且它们是使用最为广泛的TensorFlow高层封装。


### 10.2 Keras介绍

Keras API也对模型定义、损失函数、训练过程等进行了封装，而且封装之后的整个训练过程和TFLearn也基本一致，可以分为数据处理、模型定义、模型训练三个部分。  
Keras安装：`pip install keras`

**1. Keras-CNN**

以下代码展示了使用原生态Keras在MNIST数据集上实现LeNet-5模型：

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

# 1. 数据预处理
num_classes = 10
img_rows, img_cols = 28, 28
 
# 通过Keras封装好的API加载MNIST数据。其中trainX就是一个60000 * 28 * 28的数组，
# trainY是每一张图片对应的数字。
(trainX, trainY), (testX, testY) = mnist.load_data()

# 因为不同的底层（TensorFlow或MXNet）对输入的要求不一样，所以这里需要
# 根据对图像编码的格式要求来设置输入层的格式。
if K.image_data_format() == 'channels_first':
    trainX = trainX.reshape(trainX.shape[0], 1, img_rows, img_cols)
    testX = testX.reshape(testX.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    trainX = trainX.reshape(trainX.shape[0], img_rows, img_cols, 1)
    testX = testX.reshape(testX.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# 将图像像素转化为0到1之间的实数
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255.0
testX /= 255.0
 
# 将标准答案转化为需要的格式（one-hot编码）。
trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)


# 2. 使用Keras API定义模型。
model = Sequential()

# 一层深度为32，过滤器大小为5*5的卷积层
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
# 一层过滤器大小为2*2的最大池化层
model.add(MaxPooling2D(pool_size=(2, 2)))
# 一层深度为64，过滤器大小为5*5的卷积层
model.add(Conv2D(64, (5, 5), activation='relu'))
# 一层过滤器大小为2*2的最大池化层
model.add(MaxPooling2D(pool_size=(2, 2)))
# 将卷积层的输出拉直后作为下面全连接层的输入
model.add(Flatten())
# 全连接层，由500个节点
model.add(Dense(500, activation='relu'))
# 全连接层，得到最后的输出
model.add(Dense(num_classes, activation='softmax'))
 
# 定义损失函数、优化函数和评测方法。
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])


# 3. 通过Keras的API训练模型并计算在测试数据上的准确率
model.fit(trainX, trainY,
          batch_size=128,
          epochs=10,
          validation_data=(testX, testY))
 
# 在测试数据上计算准确率。
score = model.evaluate(testX, testY)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


可以看出使用Keras API训练模型过程为：
1. **先定义一个Sequential类**；
2. **然后在Sequential实例中通过add函数添加网络层**，Keras把卷积层、池化层、RNN结构（LSTM、GRN）、全连接层等常用的神经网络结构都做了封装，可以很方便地实现深层神经网络；
3. **再通过compile函数指定优化函数、损失函数以及训练过程中需要监控的指标等**，Keras对优化函数、损失函数以及监控指标都有封装，同时也支持使用自定义的方式，在Keras的API文档(https://keras.io/) 中有详细的介绍；
4. **最后Sequential实例可以通过fit函数来训练模型**，类似TFLearn中的fit函数， Keras的fit函数只须给出训练数据、batch大小和训练轮数，Keras就可以自动完成模型训练的整个过程。

**2. Keras-RNN**

Keras对于循环神经网络的支持也是非常出色的，循环体结构可以通过简单的一句命令完成。  
以下代码给出了如何通过Keras实现自然语言情感分类问题：

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

# 1. 数据预处理
max_features = 20000     # 最多使用的单词数
maxlen = 80              # 循环网络的截断长度
batch_size = 32

# 加载数据并将单词转化为ID。和自然语言模型类似，会将出现频率较低的单词替换为统一的ID。
# 通过keras封装的API会生成25000条训练数据和25000条测试数据，
# 每一条数据可以被看成一段话，并且每段话都有一个好评或者差评的标签。
(trainX, trainY), (testX, testY) = imdb.load_data(num_words=max_features)
print(len(trainX), 'train sequences')
print(len(testX), 'test sequences')

# 在自然语言中，每一段话的长度是不一样的，但循环神经网络的循环长度是固定的，
# 所以这里需要先将所有段落统一成固定长度。不够的填充0，超过的忽略超多的部分。
trainX = sequence.pad_sequences(trainX, maxlen=maxlen)
testX = sequence.pad_sequences(testX, maxlen=maxlen)
print('trainX shape:', trainX.shape)
print('testX shape:', testX.shape)


# 2. 模型定义
model = Sequential()

# 构建Embedding层，128代表了embedding向量的维度。
model.add(Embedding(max_features, 128))
# 构建LSTM层。
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# 构建最后的全连接层，注意上面在构建的LSTM层时只会得到最后一个节点的输出。
# 如果需要输出每个节点的结果，可以将return_sequence设置为True
model.add(Dense(1, activation='sigmoid'))

# 与MNIST样例类似地指定损失函数、优化方法和评估指标。
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# 3. 模型训练、测试
# 与MNIST样例类似地指定训练数据、训练轮数、batch大小以及验证数据。
model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=10,
          validation_data=(testX, testY))

score = model.evaluate(testX, testY, batch_size=batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


可以看出一个最重要地封装就是Sequential类，所有神经网络模型定义和训练都是通过这个类实现的。然而从名字就可以看出，**Sequential只支持顺序模型地定义**。**类似Inception这样的模型结构，通过Sequential类就不容易直接实现**，为了支持更加灵活的模型定义方法，**Keras支持以返回值的形式定义网络层结构**。可以参考Keras官网文档对这两种构建网络的介绍(https://keras.io/models/about-keras-models/) 。下面代码展示了如何使用这种方式定义模型：

In [ ]:
import keras
from tflearn.layers.core import fully_connected
from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model

# 1. 数据预处理
num_classes = 10
img_rows, img_cols = 28, 28
 
# 通过Keras封装好的API加载MNIST数据。
# 对比10.2.1中keras-CNN，由于这里使用全连接层，不需要将输入整理成三维矩阵
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX.reshape(trainX.shape[0], img_rows * img_cols)
testX = testX.reshape(testX.shape[0], img_rows * img_cols)
 
# 将图像像素转化为0到1之间的实数。
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255.0
testX /= 255.0
 
# 将标准答案转化为需要的格式（one-hot编码）。
trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)


# 2. 通过返回值的方式定义模型
inputs = Input(shape=(784,))

x = Dense(500, activation='relu')(inputs)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])


# 3. 训练模型
model.fit(trainX, trainY,
          batch_size=32,
          epochs=3,
          validation_data=(testX, testY))

下面代码展示如何通过Keras实现Inception结构：

In [ ]:
'''
from keras.layers import Conv2D, MaxPooling2D, Input

# 定义输入图像的尺寸
input_img = Input(shape=(256, 256, 3))

# 定义第一个分支
tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

# 定义第二个分支。与顺序模型不同，第二个分支的输入是input_img，而不是第一分支的输出
tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

# 定义第三个分支。类似的，输入也是input_img
tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

# 将三个分支通过concatenate的方式拼接在一起
output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)
'''

**2. 多输入输出**

除了可以支持非顺序模型，Keras也可以支持有多个输入或输出的模型，例如  
- 输入层1含有784个节点，代表MNIST图片中的784个像素；
- 输入层2含有10个节点，代表该图片所对应的数字；
- 输出层1在预测时仅依赖维度为1的隐藏层，因此预测的准确度较低；
- 输出层2中直接包含了正确答案，因此准确率较高。

下面代码给出了如何实现如上网络结构：

In [ ]:
import keras
from tflearn.layers.core import fully_connected
from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model

# 1. 数据预处理
num_classes = 10
img_rows, img_cols = 28, 28
 
# 通过Keras封装好的API加载MNIST数据。
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX.reshape(trainX.shape[0], img_rows * img_cols)
testX = testX.reshape(testX.shape[0], img_rows * img_cols)

trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255.0
testX /= 255.0

trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)


# 2. 定义模型
# 定义两个输入。
input1 = Input(shape=(784,), name = "input1")
input2 = Input(shape=(10,), name = "input2")

# 定义第一个输出。
x = Dense(1, activation='relu')(input1)
output1 = Dense(10, activation='softmax', name = "output1")(x)

# 定义第二个输出。
y = keras.layers.concatenate([x, input2])
output2 = Dense(10, activation='softmax', name = "output2")(y)

# 定义一个有多个输入和输出的模型，这里需要将所有的输入和输出给出即可
model = Model(inputs=[input1, input2], outputs=[output1, output2])

# 定义损失函数、优化函数和评测方法。
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              loss_weights = [1, 0.1],
              metrics=['accuracy'])


# 3. 训练模型
model.fit([trainX, trainY], [trainY, trainY],
          batch_size=128,
          epochs=3,
          validation_data=([testX, testY], [testY, testY]))

对于多输入输出需要注意的是：
- **对于损失函数**：若多个输出的损失函数相间，可以只指定一个损失函数；若多个输出的损失函数不同，则可以通过一个列表或一个字典来指定每一个输出的损失函数，比如可以使用：`loss={'output1':'binary_crossentropy', 'output2':'binary_crossentropy'｝`来为不同的输出指定不同的损失函数。

- **不同损失权重**：类似地， Keras也支持为不同输出产生的损失指定权重，这可以通过`loss_weights`参数来完成。比如在上例的定义中，output1的权重为1,output2的权重为0.1，所以这个模型会更加偏向于优化第一个输出。

- **训练的输入**：在模型训练过程，因为有两个输入和输出，所以这里提供的数据也需要有两个输入和两个期待的正确答案输出。通过列表的方式提供数据时，Keras会假设数据给出的顺序和定义Model类时输入输出给出的顺序是对应的。为了避免顺序不一致导致的问题，更推荐使用字典的形式给出：

`model.fit(
  {'input1': trainX,'input2': trainY},
  {'output1': trainY,'output2': trainY},
   ...)
`  

可以看出Keras在训练过程中会显示每个输出层的loss和accuracy。

**3. Keras + 原生态TensorFlow API**

虽然通过返回值的方式已经可以实现大部分的神经网络模型，然而Keras API还存在两大问题：
- 第一，原生态Keras API对训练数据的处理流程支持得不太好，基本上需要一次性将数据全部加载到内存；
- 第二，原生态Keras API无法支持分布式训练。

Keras提供了一种与原生态TensorFlow结合得更加紧密的方式。以下代码显示了如何联合二者来解决MNIST问题：

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist_data = input_data.read_data_sets('../../datasets/MNIST_data', one_hot=True)

# 通过TensorFlow中的placeholder定义输入。类似地，Keras封装的网络层结构也可以支持
# 使用第7章中介绍的输入队列。这样可以有效避免一次性加载所有数据的问题。
x = tf.placeholder(tf.float32, shape=(None, 784))
y_ = tf.placeholder(tf.float32, shape=(None, 10))

# 直接使用TensorFlow中提供的Keras API定义网络层结构
net = tf.keras.layers.Dense(500, activation='relu')(x)
y = tf.keras.layers.Dense(10, activation='softmax')(net)

# 定义损失函数和优化方法。注意这里可以混用Keras的API和原生态TensorFlow的API
loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_, y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 定义预测正确率作为指标
acc_value = tf.reduce_mean(tf.keras.metrics.categorical_accuracy(y_, y))

# 使用原生态TensorFlow的方式训练模型，这样可以有效实现分布式。
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(10000):
        xs, ys = mnist_data.train.next_batch(100)
        _, loss_value = sess.run([train_step, loss], feed_dict={x: xs, y_: ys})
        if i % 1000 == 0:
            print("After %d training step(s), loss on training batch is "
                  "%g." % (i, loss_value))

    test_acc = acc_value.eval(feed_dict={x: mnist_data.test.images,
                                    y_: mnist_data.test.labels})
    print('Final test accuracy is %f.' % test_acc)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
After 0 training step(s), loss on training batch is 2.40227.
After 1000 training step(s), loss on training batch is 0.0609371.
After 2000 training step(s), loss on training batch is 0.0181099.
After 3000 training step(s), loss on training ba

Keras和原生态TensorFlow紧密地结合，可以使建模的灵活性进一步提高，但是同时也会损失一部分封装带来的易用性。